<a href="https://colab.research.google.com/github/hirokimituya/book_create-profitable-AI-with-Python/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Mounted at /content/drive


In [2]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=d663131cdf8b113e18d8ba89a14ce0f73066d425bd97e04a20efa33280a34d5b
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
!pip install yahoo_finance_api2

  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3714 sha256=183be2da36b3e09820176c19eed81d2538b1049c2a486bc034ffa03811c3d33a
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [4]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [5]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    #url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    #r = requests.get(url)
    #with open('data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [6]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [7]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [8]:
import datetime as dt
import os
import pytz

jst = pytz.timezone('Asia/Tokyo')
now = dt.datetime.now(jst)
now = now.strftime("%Y%m%d_%H%M%S")
# ディレクトリ名作成（YYYYMMDD_HHMMSS_screening）
directory = f'{now}_screening'

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [9]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4262,サンドラッグ,9989,小売業
4263,サックスバー ホールディングス,9990,小売業
4264,ジェコス,9991,卸売業
4267,グローセル,9995,卸売業


In [10]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5128]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6924]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[8096]の株価情報を取得できなかったため、スキップしました。
経過時間: 170.6779716014862秒


,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1236900,643.000000
1,ＩＮＰＥＸ,1605,鉱業,6951300,1602.000000
2,安藤・間,1719,建設業,650900,1065.000000
3,コムシスホールディングス,1721,建設業,756800,2746.500000
4,オリエンタル白石,1786,建設業,520700,295.000000
...,...,...,...,...,...
338,エイチ・アイ・エス,9603,サービス業,1057500,1997.000000
339,エヌ・ティ・ティ・データ,9613,情報・通信業,3594400,1923.000000
340,ＳＣＳＫ,9719,情報・通信業,735900,2220.500000
341,ヤマダホールディングス,9831,小売業,3728700,419.899994


In [11]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1236900.0,643.000000
1,ＩＮＰＥＸ,1605,鉱業,6951300.0,1602.000000
2,安藤・間,1719,建設業,650900.0,1065.000000
3,コムシスホールディングス,1721,建設業,756800.0,2746.500000
4,大林組,1802,建設業,2472000.0,1225.500000
...,...,...,...,...,...
186,東北電力,9506,電気・ガス業,2508000.0,887.799988
187,四国電力,9507,電気・ガス業,697700.0,971.900024
188,九州電力,9508,電気・ガス業,1506900.0,915.500000
189,北海道電力,9509,電気・ガス業,784800.0,582.500000


In [12]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1236900,643.000000
1,ＩＮＰＥＸ,1605,鉱業,6951300,1602.000000
2,安藤・間,1719,建設業,650900,1065.000000
3,大林組,1802,建設業,2472000,1225.500000
4,清水建設,1803,建設業,2822000,899.599976
...,...,...,...,...,...
129,関西電力,9503,電気・ガス業,2409500,1781.000000
130,中国電力,9504,電気・ガス業,2578900,978.200012
131,北陸電力,9505,電気・ガス業,880400,766.000000
132,東北電力,9506,電気・ガス業,2508000,887.799988


In [13]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [14]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,三井住友建設,1821,建設業,879500,372.000000
1,パーソルホールディングス,2181,サービス業,852400,2610.000000
2,明治ホールディングス,2269,食料品,1016500,3266.000000
3,ベネフィット・ワン,2412,サービス業,572700,1411.500000
4,エムスリー,2413,サービス業,2437100,3006.000000
5,エスプール,2471,サービス業,8496500,637.000000
6,伊藤園,2593,食料品,535200,3827.000000
7,ユーグレナ,2931,食料品,577000,881.000000
8,ビックカメラ,3048,小売業,744700,1059.000000
9,ペッパーフードサービス,3053,小売業,502500,126.000000


In [15]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,パーソルホールディングス,2181,サービス業,852400,2610.000000
1,ベネフィット・ワン,2412,サービス業,572700,1411.500000
2,エムスリー,2413,サービス業,2437100,3006.000000
3,伊藤園,2593,食料品,535200,3827.000000
4,ビックカメラ,3048,小売業,744700,1059.000000
5,ペッパーフードサービス,3053,小売業,502500,126.000000
6,ＴＩＳ,3626,情報・通信業,507800,3559.000000
7,グリー,3632,情報・通信業,510900,612.000000
8,ＫＬａｂ,3656,情報・通信業,574400,314.000000
9,協和キリン,4151,医薬品,986900,2624.500000


In [16]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '2_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [17]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4257,タキヒヨー,9982,卸売業
4260,蔵王産業,9986,卸売業
4265,ヤマザワ,9993,小売業
4266,やまや,9994,小売業


In [18]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, max_price=5000)
filtered_tse_standard

'timestamp'
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,276600,2572.0
1,住石ホールディングス,1514,鉱業,434500,362.0
2,中小企業ホールディングス,1757,建設業,1055200,43.0
3,アジアゲートホールディングス,1783,建設業,1063300,55.0
4,土屋ホールディングス,1840,建設業,270100,233.0
...,...,...,...,...,...
93,ＲＥＶＯＬＵＴＩＯＮ,8894,不動産業,238000,15.0
94,ランド,8918,不動産業,40246100,8.0
95,ビート・ホールディングス・リミテッド,9399,情報・通信業,1337725,5.0
96,アゴーラ ホスピタリティー グループ,9704,サービス業,329900,25.0


In [19]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,434500,362.0
1,中小企業ホールディングス,1757,建設業,1055200,43.0
2,土屋ホールディングス,1840,建設業,270100,233.0
3,大盛工業,1844,建設業,266800,209.0
4,フォーサイド,2330,情報・通信業,200500,79.0
5,フュートレック,2468,サービス業,580900,316.0
6,ライフドリンク カンパニー,2585,食料品,455000,3645.0
7,イメージ ワン,2667,卸売業,2521900,716.0
8,コメ兵ホールディングス,2780,小売業,230800,4980.0
9,ビューティ花壇,3041,卸売業,457900,1037.0


In [20]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,434500,362.0
1,土屋ホールディングス,1840,建設業,270100,233.0
2,大盛工業,1844,建設業,266800,209.0
3,フォーサイド,2330,情報・通信業,200500,79.0
4,フュートレック,2468,サービス業,580900,316.0
5,イメージ ワン,2667,卸売業,2521900,716.0
6,コメ兵ホールディングス,2780,小売業,230800,4980.0
7,ＴＨＥグローバル社,3271,不動産業,695600,285.0
8,クラウディアホールディングス,3607,繊維製品,1201200,837.0
9,ジェクシード,3719,情報・通信業,495000,247.0


In [21]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '3_東証S_UPO')

In [22]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,276600,2572.0
1,アジアゲートホールディングス,1783,建設業,1063300,55.0
2,クシム,2345,情報・通信業,251600,326.0
3,出前館,2484,情報・通信業,587800,395.0
4,リミックスポイント,3825,情報・通信業,386900,220.0
5,ガーラ,4777,情報・通信業,220500,419.0
6,日本エンタープライズ,4829,情報・通信業,241400,146.0
7,エス・サイエンス,5721,非鉄金属,720400,24.0
8,三ッ星,5820,非鉄金属,1793100,2606.0
9,千代田化工建設,6366,建設業,1026400,343.0


In [23]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,276600,2572.0
1,アジアゲートホールディングス,1783,建設業,1063300,55.0
2,クシム,2345,情報・通信業,251600,326.0
3,リミックスポイント,3825,情報・通信業,386900,220.0
4,エス・サイエンス,5721,非鉄金属,720400,24.0
5,千代田化工建設,6366,建設業,1026400,343.0
6,メディアリンクス,6659,電気機器,255100,47.0
7,ピクセラ,6731,電気機器,3425300,2.0
8,キムラタン,8107,繊維製品,697000,20.0
9,プロルート丸光,8256,卸売業,243300,47.0


In [24]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '4_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [25]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
55,ルーデン・ホールディングス,1400,建設業
56,エムビーエス,1401,建設業
65,Ｌｉｂ Ｗｏｒｋ,1431,建設業
70,フィット,1436,建設業
75,ニッソウ,1444,建設業
...,...,...,...
4079,プログリット,9560,サービス業
4080,グラッドキューブ,9561,サービス業
4081,ビジネスコーチ,9562,サービス業
4082,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [26]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, max_price=5000)
filtered_tse_grows

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,232100,903.0
1,ＦＲＯＮＴＥＯ,2158,サービス業,375600,720.0
2,ジーエヌアイグループ,2160,医薬品,1238600,1262.0
3,イオレ,2334,サービス業,213300,2138.0
4,メディネット,2370,サービス業,2262400,65.0
...,...,...,...,...,...
88,サンウェルズ,9229,サービス業,259700,3310.0
89,ラバブルマーケティンググループ,9254,サービス業,3500700,1465.0
90,ｉｓｐａｃｅ,9348,サービス業,722500,1558.0
91,リニューアブル・ジャパン,9522,電気・ガス業,1611300,892.0


In [27]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,イオレ,2334,サービス業,213300,2138.0
1,タスキ,2987,不動産業,255400,1046.0
2,ｃｏｔｔａ,3359,卸売業,212400,604.0
3,リネットジャパングループ,3556,小売業,733200,649.0
4,モブキャストホールディングス,3664,情報・通信業,1496500,86.0
5,シェアリングテクノロジー,3989,情報・通信業,936900,613.0
6,ティアンドエス,4055,情報・通信業,541500,2907.0
7,プレイド,4165,情報・通信業,1105800,815.0
8,ＥＮＥＣＨＡＮＧＥ,4169,情報・通信業,290800,1336.0
9,スパイダープラス,4192,情報・通信業,265400,802.0


In [28]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,イオレ,2334,サービス業,213300,2138.0
1,ｃｏｔｔａ,3359,卸売業,212400,604.0
2,リネットジャパングループ,3556,小売業,733200,649.0
3,モブキャストホールディングス,3664,情報・通信業,1496500,86.0
4,ティアンドエス,4055,情報・通信業,541500,2907.0
5,Ｆｉｎａｔｅｘｔホールディングス,4419,情報・通信業,253900,835.0
6,免疫生物研究所,4570,医薬品,12064300,652.0
7,ストリームメディアコーポレーション,4772,情報・通信業,583500,284.0
8,レナサイエンス,4889,医薬品,826200,516.0
9,ＱＤレーザ,6613,電気機器,2891500,811.0


In [29]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '5_東証G_UPO')

In [30]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,232100,903.0
1,メディネット,2370,サービス業,2262400,65.0
2,サンワカンパニー,3187,小売業,350900,672.0
3,ディー・ディー・エス,3782,情報・通信業,23117200,16.0
4,グローバルウェイ,3936,情報・通信業,227500,219.0
5,ココナラ,4176,情報・通信業,330500,340.0
6,アクリート,4395,情報・通信業,463400,1395.0
7,ＢＡＳＥ,4477,情報・通信業,1049300,261.0
8,アンジェス,4563,医薬品,546300,107.0
9,オンコセラピー・サイエンス,4564,医薬品,3545300,38.0


In [31]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,メディネット,2370,サービス業,2262400,65.0
1,ディー・ディー・エス,3782,情報・通信業,23117200,16.0
2,ココナラ,4176,情報・通信業,330500,340.0
3,ＢＡＳＥ,4477,情報・通信業,1049300,261.0
4,リボミック,4591,医薬品,221900,156.0
5,サンバイオ,4592,医薬品,342700,610.0
6,ヘリオス,4593,医薬品,9409000,289.0
7,窪田製薬ホールディングス,4596,医薬品,226600,101.0
8,トレンダーズ,6069,サービス業,224200,1056.0
9,アライドアーキテクツ,6081,サービス業,232100,410.0


In [32]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [33]:
import pandas as pd

def remove_duplicate_rows(dataframe, *other_dataframes):
    """重複しているティッカーコードの行を削除する

    :param dataframe: 対象のデータフレーム
    :param other_dataframes: 重複しているかどうかの判断のためのデータフレーム
    :return 引数dataframeで重複行を削除したもの
    """
    # データフレームをコピーして新しいデータフレームを作成
    result_df = dataframe.copy()

    # 重複確認のためのセットを作成
    unique_values = set()
    for df in other_dataframes:
        unique_values.update(df['コード'])

    # 重複しているティッカーコードを削除
    result_df = result_df[~result_df['コード'].isin(unique_values)]

    return result_df

In [34]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8521]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,中小企業ホールディングス,1757,建設業,1055200,43.0
1,アジアゲートホールディングス,1783,建設業,1063300,55.0
2,燦キャピタルマネージメント,2134,サービス業,12519500,26.0
3,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,1494800,59.0
4,フュートレック,2468,サービス業,580900,316.0
5,出前館,2484,情報・通信業,587800,395.0
6,ベクター,2656,小売業,523100,324.0
7,イメージ ワン,2667,卸売業,2521900,716.0
8,ピクセルカンパニーズ,2743,卸売業,918200,48.0
9,ＴＨＥグローバル社,3271,不動産業,695600,285.0


In [35]:
# ティッカーコードが重複している行を削除
filtered_tse_standard_2_remove_duplicate = remove_duplicate_rows(filtered_tse_standard_2, filtered_perfect_order_week_day_tse_standard, filtered_reverse_perfect_order_week_day_tse_standard)
filtered_tse_standard_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
0,中小企業ホールディングス,1757,建設業,1055200,43.0
2,燦キャピタルマネージメント,2134,サービス業,12519500,26.0
3,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,1494800,59.0
5,出前館,2484,情報・通信業,587800,395.0
6,ベクター,2656,小売業,523100,324.0
8,ピクセルカンパニーズ,2743,卸売業,918200,48.0
10,レカム,3323,卸売業,663900,86.0
12,ｅｎｉｓｈ,3667,情報・通信業,858600,341.0
13,ディー・エル・イー,3686,情報・通信業,779400,264.0
14,ブロードバンドタワー,3776,情報・通信業,1672300,153.0


In [36]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2_remove_duplicate, '7_東証S_選抜')

In [37]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_grows_2 = filtered_ticker(tse_grows, min_volume=500000, max_price=5000)
filtered_tse_grows_2

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,1238600,1262.0
1,メディネット,2370,サービス業,2262400,65.0
2,ウェッジホールディングス,2388,その他金融業,3317200,138.0
3,フルッタフルッタ,2586,食料品,1078000,85.0
4,リネットジャパングループ,3556,小売業,733200,649.0
5,モブキャストホールディングス,3664,情報・通信業,1496500,86.0
6,デジタルプラス,3691,情報・通信業,625100,787.0
7,ディー・ディー・エス,3782,情報・通信業,23117200,16.0
8,Ａｉｍｉｎｇ,3911,情報・通信業,1043800,435.0
9,ソーシャルワイヤー,3929,情報・通信業,1060500,384.0


In [38]:
# ティッカーコードが重複している行を削除
filtered_tse_grows_2_remove_duplicate = remove_duplicate_rows(filtered_tse_grows_2, filtered_perfect_order_week_day_tse_grows, filtered_reverse_perfect_order_week_day_tse_grows)
filtered_tse_grows_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
0,ジーエヌアイグループ,2160,医薬品,1238600,1262.0
2,ウェッジホールディングス,2388,その他金融業,3317200,138.0
3,フルッタフルッタ,2586,食料品,1078000,85.0
6,デジタルプラス,3691,情報・通信業,625100,787.0
8,Ａｉｍｉｎｇ,3911,情報・通信業,1043800,435.0
9,ソーシャルワイヤー,3929,情報・通信業,1060500,384.0
10,シェアリングテクノロジー,3989,情報・通信業,936900,613.0
12,プレイド,4165,情報・通信業,1105800,815.0
13,エクサウィザーズ,4259,情報・通信業,2512800,487.0
15,スペースマーケット,4487,情報・通信業,803800,568.0


In [39]:
# CSVファイル保存
save_df_as_csv(filtered_tse_grows_2_remove_duplicate, '8_東証G_選抜')